In [4]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [5]:
import pandas as pd 
import numpy as np
import time
from sentence_transformers import SentenceTransformer,util
import tensorflow
import pickle

In [8]:
from fobokiller.cosim import load_embedding , compute_sim_df ,model

In [13]:
embedding = load_embedding()

2021-11-26 15:28:39.804607: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [ ]:
df = compute_sim_df('I want to eat a pizza in a cosy place',np.array(embedding), 2000)
df['ratio']=df['review_filtered']/df['review_all']
df

In [ ]:
df['ratio_x_sim'] = df['ratio']*df['sim_r']
df['ponderation_review'] = df['ratio']*df['sim_r']*df['review_all']
df['ponderation_rate_review'] = df['ratio']*df['sim_r']*df.rate_all/df.review_all

In [ ]:
df.groupby('alias').sum().sort_values(by = 'ponderation_rate_review').index

In [ ]:
ponderation_rate_review = df.groupby('alias').sum().sort_values(by = 'ponderation_rate_review', 
                                      ascending=False)[:20].index

ponderation_review = df.groupby('alias').sum().sort_values(by = 'ponderation_review', 
                                      ascending=False)[:20].index


ratio = df.groupby('alias').sum().sort_values(by = 'ratio', 
                                      ascending=False)[:20].index
sim = df.groupby('alias').sum().sort_values(by = 'sim_r', 
                                      ascending=False)[:20].index
                                                                              

In [ ]:
pd.DataFrame([sim,ratio,ponderation_rate, ponderation_review, ponderation_rate_review ], 
             index = ["sim","ratio", "ponderation_review", "ponderation_rate_review"]).T